In [1]:
import netCDF4 as nc
import numpy as np


def local_search(target_lat, target_lon):
    # NetCDF 파일 열기
    file_path = './data/KMAPP_solar_FWS_01M_mean.nc'
    dataset = nc.Dataset(file_path, 'r')

    # X (경도)와 Y (위도) 변수 읽기
    x = dataset.variables['X'][:]  # 경도
    y = dataset.variables['Y'][:]  # 위도

    # 좌표에 가장 가까운 경도(X)와 위도(Y)의 인덱스 찾기
    x_idx = np.abs(x - target_lon).argmin()
    y_idx = np.abs(y - target_lat).argmin()

    # # 파일 닫기
    dataset.close()

    return x_idx, y_idx


# # 각 변수에서 해당 좌표의 데이터 가져오기
# for var_name in dataset.variables:
#     variable = dataset.variables[var_name]
#     if 'X' not in var_name and 'Y' not in var_name:  # X, Y는 건너뛰기
#         monthly_solar_radiation.append(round(float(variable[y_idx, x_idx]),2))
#         print(f"{var_name}에서 ({target_lat}, {target_lon}) 수평면의 일사량_01M : {monthly_solar_radiation[0]}"

In [2]:
import netCDF4 as nc

def process_monthly_data(file_path, y_idx, x_idx, monthly_solar_radiation):
    # NetCDF 파일 열기
    dataset = nc.Dataset(file_path, 'r')
    
    # 변수들을 순회하면서 X, Y를 제외하고 데이터를 처리
    for var_name in dataset.variables:
        variable = dataset.variables[var_name]
        if 'X' not in var_name and 'Y' not in var_name:  # X, Y는 건너뛰기
            # 여기서 y_idx와 x_idx를 올바르게 인덱싱
            if variable.ndim == 2:  # 2차원일 경우
                monthly_solar_radiation.append(round(float(variable[y_idx, x_idx]), 2))
            elif variable.ndim > 2:  # 3차원 이상의 경우 첫 번째 시간 축(또는 다른 축)을 처리
                monthly_solar_radiation.append(round(float(variable[0, y_idx, x_idx]), 2))  # 시간 축이 있을 경우 첫 번째 슬라이스만 사용

    # 파일 닫기
    dataset.close()

# 파일 경로 목록
file_paths = [
    './data/KMAPP_solar_FWS_01M_mean.nc',
    './data/KMAPP_solar_FWS_02M_mean.nc',
    './data/KMAPP_solar_FWS_03M_mean.nc',
    './data/KMAPP_solar_FWS_04M_mean.nc',
    './data/KMAPP_solar_FWS_05M_mean.nc',
    './data/KMAPP_solar_FWS_06M_mean.nc',
    './data/KMAPP_solar_FWS_07M_mean.nc',
    './data/KMAPP_solar_FWS_08M_mean.nc',
    './data/KMAPP_solar_FWS_09M_mean.nc',
    './data/KMAPP_solar_FWS_10M_mean.nc',
    './data/KMAPP_solar_FWS_11M_mean.nc',
    './data/KMAPP_solar_FWS_12M_mean.nc'
]


<h2> 함수 solar_radiation </h2>
<h2> 입력은 좌표(lat, lon)이고 출력은 월별 일사량입니다 </h2>

In [3]:
import pandas as pd


def solar_radiation(target_lat, target_lon):
    monthly_solar_radiation=[]

    x_idx, y_idx = local_search(target_lat, target_lon)

    # 파일 경로를 순회하면서 데이터를 처리
    for file_path in file_paths:
        process_monthly_data(file_path, y_idx, x_idx, monthly_solar_radiation)

    df_monthly_solar_radiation = pd.DataFrame({
        '월' : ['01M','02M','03M','04M','05M','06M','07M','08M','09M','10M','11M','12M'],
        '수평면_일사량_GHI' : monthly_solar_radiation
    })
    return df_monthly_solar_radiation

In [14]:
# 주어진 좌표
target_lat = 35.8567148
target_lon = 128.4949977
df_monthly_solar_radiation = solar_radiation(target_lat, target_lon)
df_monthly_solar_radiation

,월,수평면_일사량_GHI
0,01M,185.69
1,02M,255.39
2,03M,338.65
3,04M,393.53
4,05M,392.27
5,06M,362.80
6,07M,382.81
7,08M,400.98
8,09M,299.78
9,10M,269.58


<h3>대구 지역의 일사량만 가져오려고했는데 처리시간이 너무 오래 걸려서 포기함 </h3>

In [ ]:
# import netCDF4 as nc
# import numpy as np
# import pandas as pd

# # NetCDF 파일 열기
# file_path = './data/KMAPP_solar_FWS_01M_mean.nc'
# dataset = nc.Dataset(file_path, 'r')

# # 경도(X), 위도(Y), 일사량(SWDN_flat_with_shading) 변수 가져오기
# x = dataset.variables['X'][:]  # 경도
# y = dataset.variables['Y'][:]  # 위도
# swdn_flat_with_shading = dataset.variables['SWDN_flat_with_shading'][:, :]  # 일사량

# # DataFrame 생성
# # 데이터프레임은 X와 Y의 그리드 구조에서 각 지점의 일사량을 기록합니다.
# data_list = []
# for i in range(len(y)):  # 위도
#     for j in range(len(x)):  # 경도
#         data_list.append([y[i], x[j], swdn_flat_with_shading[i, j]])

# # DataFrame으로 변환
# df = pd.DataFrame(data_list, columns=['Latitude', 'Longitude', 'SWDN_flat_with_shading'])

# # 데이터프레임 정보 출력
# print(df.head())  # 상위 5개 데이터 출력

# # NetCDF 파일 닫기
# dataset.close()
